In [1]:
import os
import pandas as pd
import json

In [117]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### 데이터 불러오기

#### 1. 가사

In [109]:
lyrics1 = pd.read_csv("./dataset/label_result_song_short.csv")
lyrics2 = pd.read_csv("./dataset/translated_lyrics.csv")

lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])

#### 2. 소설

In [ ]:
novel_final = pd.read_csv('./dataset/novel_final.csv')
data = novel_final.copy()

### Preprocessing

#### 1. 가사

In [118]:
# 감정 카테고리
emotions = ['상처', '불안', '기쁨', '슬픔', '분노', '당황']

In [113]:
# 각 id에 대해 sentiment_label의 빈도를 계산하여, 가장 많이 나온 2개의 감정을 새로운 칼럼으로 추가

def get_top_two_sentiments(sentiments):
    sentiment_counts = sentiments.value_counts()
    # 가장 많은 2개 감정을 반환
    top_two = sentiment_counts.head(2).index.tolist()
    # 2개 미만일 경우 빈 값 처리
    return top_two + [None] * (2 - len(top_two))

# groupby와 agg를 활용해 변환
lyrics_final = lyrics.groupby("id").agg({
    "title": "first",   # 첫 번째 값 유지
    "singer": "first",  # 첫 번째 값 유지
    "genres": "first",  # 첫 번째 값 유지
    "lyrics": "first",  # 첫 번째 값 유지
    "translated_lyrics": list,  # 리스트로 묶음
    "sentiment_label": lambda x: get_top_two_sentiments(x)  # 빈도가 높은 2개의 감정 추출
}).reset_index()

# sentiment_label에서 두 개의 감정을 각각의 칼럼으로 분리
lyrics_final[['top_sentiment', 'second_sentiment']] = pd.DataFrame(lyrics_final['sentiment_label'].tolist(), index=lyrics_final.index)

# 불필요한 sentiment_label 열 삭제
lyrics_final = lyrics_final.drop(columns=["sentiment_label"])

In [114]:
lyrics_final.head()

,id,title,singer,genres,lyrics,translated_lyrics,top_sentiment,second_sentiment
0,418168,희재,성시경,"발라드, 국내영화",햇살은 우릴 위해 내리고,"[햇살은 우릴 위해 내리고 , 바람도 서롤 감싸게 했죠 , 우리 웃음속에, 계절은 ...",슬픔,기쁨
1,418598,친구라도 될 걸 그랬어,거미 (GUMMY),R&B/Soul,벌써 넌 내가 편하니,"[벌써 넌 내가 편하니, 웃으며 인사 할 만큼, 까맣게 나를 잊었니, 네 곁에 있는...",불안,슬픔
2,711626,살다가,SG 워너비,발라드,살아도 사는 게 아니래,"[살아도 사는 게 아니래, 너 없는 하늘에, 창 없는 감옥 같아서, 웃어도 웃는 게...",슬픔,불안
3,1500196,내사람,SG 워너비,R&B/Soul,내 가슴속에 사는 사람 내가 그토록 아끼는 사람,"[내 가슴속에 사는 사람 내가 그토록 아끼는 사람 , 너무 소중해 마음껏 안아보지도...",기쁨,불안
4,1854856,라라라,SG 워너비,발라드,그대는 참 아름다워요,"[그대는 참 아름다워요, 밤 하늘의 별빛보다 빛나요, 지친 나의 마음을 따뜻하게 감...",기쁨,슬픔


In [119]:
le = LabelEncoder()
le.fit(emotions)

lyrics_final['감정1_encoded'] = le.transform(lyrics_final['top_sentiment'])
lyrics_final['감정2_encoded'] = le.transform(lyrics_final['second_sentiment'])

#### 2. 소설

In [95]:
data.head()

,Unnamed: 0,title,text,top_sentiment,second_sentiment
0,0,이야기꾼 구연설화,"['01범보다 무서운 곶감', '화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴...",분노,당황
1,1,이야기꾼 구연설화,"['<봄꿩은 제 울음에 저 죽는다>', '그 말과 같아서 사램이 잘못 되머넌 하는 ...",상처,분노
2,2,이야기꾼 구연설화,"['그래 그 여자가 쪄서 쌀얼 쪄서 밥얼 했어.', '“잡수라.”구.', '그래 한...",상처,당황
3,3,이야기꾼 구연설화,"['그랴. 그래 우리 인제 사춘 찾어간다는 얘기를 족~ 갈쳐중개,', '“그러시냐구...",상처,분노
4,4,이야기꾼 구연설화,"['나와서 인제 그 집 먼지 인저 그, 그, 뭐여 면사무소 있는 디 나와서 인제, ...",상처,분노


In [ ]:
# LabelEncoder 초기화
le = LabelEncoder()
le.fit(emotions)

# 감정1과 감정2를 숫자로 변환
data['감정1_encoded'] = le.transform(data['top_sentiment'])
data['감정2_encoded'] = le.transform(data['second_sentiment'])

# 수치형 감정1, 감정2와 텍스트 결합
data['input_text'] = data['감정1_encoded'].astype(str) + " | " + data['감정2_encoded'].astype(str) + " | " + data['text']

# 불필요한 문자 제거
data['input_text'] = data['input_text'].str.replace(r"[\[\],']", "", regex=True)

data[['top_sentiment', 'second_sentiment', 'input_text']].sample(n=5)

,top_sentiment,second_sentiment,input_text
2400,불안,슬픔,3 | 5 | 18부 영우는 차창을 바라보았다. 찬바람이 여러 겹씩 두껍게 옷을 입...
6877,불안,기쁨,3 | 0 | 의문과 실망으로 착잡한 박전전과는 달리 서천양은 여유만만이었다. 입이...
5369,불안,슬픔,3 | 5 | 규하가 춘봉을 향해 뚜벅뚜벅 걸어왔다. 춘봉의 동공이 점점 커지고 있...
4268,기쁨,불안,0 | 3 | 엘프답지 않은 쌍스러운 말이었지만 일반적인 엘프와는 달리 전사의 기운...
1326,불안,당황,"3 | 1 | ""사실 우연이란 놈은 무서운 장난꾼입니다. 저는 이 장난꾼에게 한번 ..."


In [97]:
# 감정별 숫자 매핑 출력
for emotion, label in zip(le.classes_, le.transform(le.classes_)):
    print(f"{emotion}: {label}")

기쁨: 0
당황: 1
분노: 2
불안: 3
상처: 4
슬픔: 5


In [98]:
# 데이터셋 클래스 정의
class NovelsDataset(Dataset):
    def __init__(self, data, tokenizer, text_column="input_text", max_length=128):
        self.data = data.reset_index(drop=True)  # 인덱스 리셋 (중복 방지)
        self.tokenizer = tokenizer
        self.text_column = text_column
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.loc[idx, self.text_column] # input_text 가져오기

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        # labels 생성 (input_ids 복사 및 패딩 토큰을 -100으로 설정)
        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
        }

### Modeling

In [99]:
# KOGPT2 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

# 모델 초기화
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.bos_token_id = tokenizer.bos_token_id

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [100]:
dataset = NovelsDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=2)

In [101]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    save_steps=50,
    save_total_limit=2,
    logging_steps=10,
    eval_strategy="epoch",
)

In [102]:
train_novels, eval_novels = train_test_split(data, test_size=0.1, random_state=42)

train_dataset = NovelsDataset(train_novels, tokenizer)
eval_dataset = NovelsDataset(eval_novels, tokenizer)

In [103]:
# Trainer에 데이터셋 전달
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # 검증 데이터셋
)

In [104]:
sample = train_dataset[0]
print("Input IDs:", sample['input_ids'])
print("Attention Mask:", sample['attention_mask'])
print("Labels:", sample['labels'])

Input IDs: tensor([ 9085,   739,   466,  9020,   739,   466, 10063,  7584,  9391, 21182,
        12311, 35142, 13875,  9317, 16299, 13568, 18017,  9717, 17631, 11247,
        15972,  7621,  7422,  8263,   739,     5, 23775, 47711, 10063,  7669,
         7445, 11001, 22580, 26309,  7185,  9135, 13115,   406,   377,  9546,
         7407, 32552,  9069,  7560,  8159, 22301,  8024,  9274,  7407,  7543,
         8146, 16626, 13872, 10063,  8762, 10072,  9215, 18282,  9215,  9878,
        13969,  9261, 19023,  9174,  9179,  9432, 12004,  9825, 12024,  7281,
         6883,  9173, 13444, 32987,   377, 10063,  8052,  9055,  8694, 15509,
          376,  9174,  9179,  9432, 17104,  8744,  7055,  7652,  6919,  7514,
          389, 46651,  6824, 12222, 21832,  9122,  7991,   406, 25758,  9720,
         6958,  9237, 22882,   406, 16518, 15518, 39717,  9193, 22882,   406,
          377,  9546,  7407, 19902, 18191,  6824, 32738,  8006,  9042,  9313,
        44083, 14870,  9064,  7472,  7326,  9016, 100

In [105]:
# 모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.206400,4.094640
2,3.811800,4.055212
3,3.500900,4.078753
4,2.900000,4.142436


TrainOutput(global_step=5045, training_loss=3.507166806911926, metrics={'train_runtime': 1298.1008, 'train_samples_per_second': 31.099, 'train_steps_per_second': 3.886, 'total_flos': 2634868850688000.0, 'train_loss': 3.507166806911926, 'epoch': 4.995788952192222})

In [124]:
device = torch.device("cuda")

# 이야기 생성 함수
def generate_story(lyrics_input):
    # 입력 토큰화
    input_ids = tokenizer.encode(lyrics_input, return_tensors='pt').to(device)

    # 출력 생성
    output = model.generate(
        input_ids, 
        max_new_tokens=150, # 새로 생성할 토큰의 개수를 제한
        num_return_sequences=1, 
        temperature=0.8, 
        top_k=50, 
        top_p=0.9, 
        repetition_penalty=1.2, 
        do_sample=True  # 샘플링 활성화
    )

    # 입력 길이 추적
    input_length = input_ids.shape[1]

    # 생성된 토큰 중 입력 토큰 이후의 부분만 디코딩
    generated_tokens = output[0][input_length:]
    generated_story = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return generated_story

In [156]:
i = 732
print(lyrics_final["title"][i])

Welcome to the Show


In [157]:
lyrics = " ".join(lyrics_final["translated_lyrics"][i])

prompt = lyrics_final['감정1_encoded'][i].astype(str) + " | " + lyrics_final['감정2_encoded'][i].astype(str) + " | " + lyrics
print(prompt)

3 | 0 | 이젠 혼자가 아닐 무대 너무나 감격스러워 끝없는 가능성 중에 날 골라줘서 고마워 나와 맞이하는 미래가 위태로울지도 몰라 하지만 눈물 가득한 감동이 있을지도 몰라 그래도 내 손 놓지 않겠다면 만약 그래서 그러면 봅시다 밖으로 나가  환영합니다. 옆으로 그리고 보여주는 것 Oh 이것만큼은 맹세할게 내 전부를 다 바칠게 네 눈빛 흔들리지 않게 널 바라보며 서 있을게 알아 너의 결정이  쉽지 않았을 거야 후회 없게 하는 건  이제 나의 몫이야 끝까지 같이 함께 가겠다면 만약 그래서 그러면 봅시다 밖으로 나가 환영합니다. 옆으로 그리고 보여주는 것 Oh 이것만큼은 맹세할게 내 전부를 다 바칠게 네 눈빛 흔들리지 않게 널 바라보며 서 있을게 막이 내릴 그날에도 그때도 네 손 꼭 잡은 채 너라서 행복했다고 서로가 말할 수 있도록  이것만큼은 맹세할게 내 전부를 다 바칠게 네 눈빛 흔들리지 않게 널 바라보며 서 있을게 Oh Oh


In [158]:
# 가사로 이야기 생성
generated_story = generate_story(prompt)
print("생성된 이야기:", generated_story)

생성된 이야기: ! "와아~-씨아. 안녕하하셨어요?" 나 오랜만에 좋은 아침인데 반갑습니다 반가운 마음에 드는 얼굴로 인사합니다 우린다. 어제 저녁에 잘 먹고 인사 나누었는데 너무 좋네. 축하해 우리 윤정이가 간만에 하이라며 인사하러 갑니가 먼저 가자꾸나~^. 흐흐아 정혜리 오후의 두근덕한 기분 좋아하네 미팅으로 반갑에 대한 인사를 받고 집으로 가시고 정숙인께 우리 집에 들러 앉으니까 하여 차 한잔하게 저녁 먹자.... "윤아~오케 하기로 합심해서 인사해주세." "와하네. 아이고우~~ ᆞ0_


In [ ]:
# 가사로 이야기 생성
# lyrics_input = "어쩌다 고작 그 마음도 못 참고 멍청하게 다 던졌는지 뭔가 들켜 버린 것 같아 표정을 보니 말이야 나도 티가 나버린 고백에 얼마나 놀랐는지 몰라 매일 치는 장난에도 두근댔고 오늘도 몇 번이고 떨렸지만 약속했어 날 안아줘 좀 알아줘 이건 꿈에서만 하기야 무심코 던진 니 말에 하루 종일 설레어 간직했다 아무도 못 보게 일기장에 적어 단단히 잠궜었는데 어쩌다 고작 그 마음도 못 참고 멍청하게 다 던졌는지 꾹꾹 참고 또 꼭꼭 숨겨서 이제까지 잘해 왔잖아 그러다 고작 울음도 못 참고 괜찮다 말하며 두 눈은 퉁퉁 붓고 코맹맹이가 되어도 난 내일은 맑음 예전처럼 옆에서 밥 먹어도 우연히 눈이 살짝 마주쳐도 걱정 마 날 안아줘 아니 사랑해줘 이건 꿈에서만 하니까 무심코 던진 니 말에 하루 종일 설레어 간직했다 아무도 못 보게 꼬깃꼬깃 구겨 씹어 다 삼켰었는데 어쩌다 고작 그 마음도 못 참고 멍청하게 다 던졌는지 꾹꾹 참고 또 꼭꼭 숨겨서 이제까지 잘 해 왔잖아 그러다 고작 울음도 못 참고 괜찮다 말하며 두 눈은 퉁퉁 붓고 코맹맹이가 되어도 난 사실 나 아주 오래 울 것 같아 고작 친구도 못 되니까 툭툭 털고 활짝 웃을 만큼 나는 그리 강하지가 않아 그러다 고작 사랑이 뭐라고 괜찮다 말하는 날까지 꾹꾹 참고 또 일기나 쓰고 있어 나 내 이름 맑음"
# generated_story = generate_story(lyrics_input, tag1="2", tag2="2")
# print("생성된 이야기:", generated_story)

생성된 이야기: 인지 이젠데 왜 내가 좋아하는지 그래 속상하기만 말예 퇴근거리는 게 참으론 정말이지 남이 나를 버리고 지금이 딱히 한숨 쉬었는지 알면서.... 이러면 금방엔 숨소리 질러 가며 소리 죽여 버릇한 기분이 좋아서 그냥 그러는 거냐? 이제 더러운 세상 물끄떡하고 예사로운 걸어서 그런 건가 못해~ 하고 사는데. 그만 이렇게 예쁜 척하는 소리가 나랑말루 오죽 나래 그래 가지 않고 그저 눈 감고 삐딱다리 놓으면 살그렁거니 이마에 이어두 못 하게 할까 봐놓고 사는 저러다 말고 다리 뻗고 가만히 들여다보면 예뻐한다ᆞ오래야 제자리 앉으며 맘
